In [0]:
#Part 3
#Group 291

In [0]:
 #The Data preprocessing in for this process is similar to the clustering one up until the reading of the file. Therefore that is not shown here in this notebook.
#Once the file is read using File_Reader function it is put into a data frame.
#File reader function will read the JSON files and put it into a list which is of the form--dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}

  
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

,paper_id,abstract,body_text,authors,title,journal,abstract_summary
0,be89ef4d5920eebd0d113c5904a202a4e95cc41d,A pro-active emerging risk identification syst...,Food safety has ameliorated over the years due...,"van Asselt, E.D.. Meuwissen, M.P.M....",Selection of critical factors for identifying...,Food Control,A pro-active emerging risk identification<br>...
1,da214d4b0e7f7bb7d851fe7d2d38b30989b097fe,Microorganisms are very efficient at infecting...,Epidemiology is the study of how disease is di...,"MORSE, STEPHEN A.. KHAN, ALI S.",CHAPTER 8 Epidemiologic Investigation for<br>...,Microbial Forensics,Microorganisms are very efficient at<br>infec...
2,5534334dbd5c0b7150ecd803ea4f3f17a3da6369,,Dieses Kapitel befasst sich mit den Übertragun...,"Marino, Paul L.",Kapitel 4 Berufsbedingte Expositionen,Das ICU-Buch,Not provided.
3,788d03b2321d5e60924c901d1f57a14cb081aad9,,Middle East respiratory syndrome corona virus ...,"Mohamed, Deqa H.. AlHetheel, AbdulKarim F....",Clinical validation of 3 commercial real-time...,Diagnostic Microbiology and Infectious Disease,Not provided.
4,85f5968642411598e44291f2a10df85101b8799f,A solid phase colloidal gold immunoelectron mi...,Neonatal calf diarrhoea is a syndrome of compl...,"El-Ghorr, A.A.. Snodgrass, D.R....",Evaluation of an immunogold electron<br>micro...,Journal of Virological Methods,A solid phase colloidal gold immunoelectron<b...


In [0]:
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

100%|██████████| 12500/12500 [00:00<00:00, 27295.09it/s]


In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("/content/covid_sentences.csv")
df_sentences.head()

,paper_id
food safety has ameliorated over the years due to the application of hazard analysis critical control points haccp systems and the development of risk assessments ra haccp is a systematic approach to the identification evaluation and control of those steps in food manufacturing that are critical to product safety the basic objective of the haccp concept is assuring the production of safe food products by prevention instead of quality inspection luning marcelis jongen 2002 haccp is a system applied to identify known food safety hazards and is currently applied per stage in the production supply chain instead of the total production chain food safety ra comprises the scientific evaluation of known or potential adverse health effects resulting from human exposure to specific food borne hazards codex 1999 it typically uses data on the particular hazard and production chain under consideration and modelling to estimate the final likelihood of harm due to human exposure both haccp and ra focus on known hazards and make use of historical data related to the particular hazards as well as to the particular chain of interest risk managers and assessors need to get access to all available data on food safety hazards as soon as possible for this purpose there are various warning systems for notification on the likelihood of a hazard examples are the eu rapid alert system on food and feed rasff httpeceuropaeufoodfoodrapidalertindex_enhtm the whoglobal outbreak alert and response network http wwwwhointcsroutbreaknetworken and the global public health intelligence network gphin in canada httpwww phacaspcgccamedianrrp20042004_gphinrmispbk_ehtml such warning systems address known wellcharacterized food and feed safety hazards vwa 2006 in order to identify and prevent emerging hazards leading to food safety risks it is necessary to move towards a more proactive system for identification of emerging food and feed related risks an emerging risk er is defined as a risk resulting from a newly identified hazard to which a significant exposure may occur or from an unexpected new or increased significant exposure andor susceptibility to a known hazard efsa 2007 er may be directly linked to the food production chain or indirectly connected to it therefore in order to identify emerging risks in an early stage an holistic approach is proposed vwa 2005 this approach implies that emergence of a risk can be the result from factors inside the production chain endogenous or outside the chain exogenous in addition emergence of hazards related to risks is usually a result of a particular change inside or outside the production chain a proactive system for the identification of emerging food safety risks should therefore preferable be based on endogenous and exogenous factors characterizing the dynamics of a food production system endogenous factors associated with changes within the production chain may be related to technological innovations their implementation driven from production perspectives exogenous factors associated with changes outside the production chain may include for example economic changes climate change international trade and changes in human behaviour these exogenous factors are related to influential sectors as described in the emrisk and periapt project vwa 2005 vwa 2006,be89ef4d5920eebd0d113c5904a202a4e95cc41d
once critical factors are identified signals can be established that indicate directly or indirectly the occurrence of an emerging hazard the socalled indicators an example of a critical factor is climate change with temperature and rainfall as indicators van der felsklerx kandhai booij 2008 vwa 2005 these indicators form the key elements of an er identification system for these key indicators information sources data and expertise and critical limits should be determined when the limit of one or more indicators is exceeded required actions can be taken in an early stage to prevent food safety problems occurring as a res

In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("/content/covid_sentences_Full.csv")
df_sentences.head()


,paper_id,title,abstract,abstract_summary
food safety has ameliorated over the years due to the application of hazard analysis critical control points haccp systems and the development of risk assessments ra haccp is a systematic approach to the identification evaluation and control of those steps in food manufacturing that are critical to product safety the basic objective of the haccp concept is assuring the production of safe food products by prevention instead of quality inspection luning marcelis jongen 2002 haccp is a system applied to identify known food safety hazards and is currently applied per stage in the production supply chain instead of the total production chain food safety ra comprises the scientific evaluation of known or potential adverse health effects resulting from human exposure to specific food borne hazards codex 1999 it typically uses data on the particular hazard and production chain under consideration and modelling to estimate the final likelihood of harm due to human exposure both haccp and ra focus on known hazards and make use of historical data related to the particular hazards as well as to the particular chain of interest risk managers and assessors need to get access to all available data on food safety hazards as soon as possible for this purpose there are various warning systems for notification on the likelihood of a hazard examples are the eu rapid alert system on food and feed rasff httpeceuropaeufoodfoodrapidalertindex_enhtm the whoglobal outbreak alert and response network http wwwwhointcsroutbreaknetworken and the global public health intelligence network gphin in canada httpwww phacaspcgccamedianrrp20042004_gphinrmispbk_ehtml such warning systems address known wellcharacterized food and feed safety hazards vwa 2006 in order to identify and prevent emerging hazards leading to food safety risks it is necessary to move towards a more proactive system for identification of emerging food and feed related risks an emerging risk er is defined as a risk resulting from a newly identified hazard to which a significant exposure may occur or from an unexpected new or increased significant exposure andor susceptibility to a known hazard efsa 2007 er may be directly linked to the food production chain or indirectly connected to it therefore in order to identify emerging risks in an early stage an holistic approach is proposed vwa 2005 this approach implies that emergence of a risk can be the result from factors inside the production chain endogenous or outside the chain exogenous in addition emergence of hazards related to risks is usually a result of a particular change inside or outside the production chain a proactive system for the identification of emerging food safety risks should therefore preferable be based on endogenous and exogenous factors characterizing the dynamics of a food production system endogenous factors associated with changes within the production chain may be related to technological innovations their implementation driven from production perspectives exogenous factors associated with changes outside the production chain may include for example economic changes climate change international trade and changes in human behaviour these exogenous factors are related to influential sectors as described in the emrisk and periapt project vwa 2005 vwa 2006,be89ef4d5920eebd0d113c5904a202a4e95cc41d,Selection of critical factors for identifying...,a proactive emerging risk identification syste...,A pro-active emerging risk identification<br>...
once critical factors are identified signals can be established that indicate directly or indirectly the occurrence of an emerging hazard the socalled indicators an example of a critical factor is climate change with temperature and rainfall as indicators van der felsklerx kandhai booij 2008 vwa 2005 these indicators form the key elements of an er identification system for these key indicators information sources data and expertise and critical

In [0]:

df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]
print(len(df_sentences_list))
df_sentences_list[:5]

100%|██████████| 429177/429177 [00:00<00:00, 2038226.89it/s]

429177


['food safety has ameliorated over the years due to the application of hazard analysis critical control points haccp systems and the development of risk assessments ra haccp is a systematic approach to the identification evaluation and control of those steps in food manufacturing that are critical to product safety the basic objective of the haccp concept is assuring the production of safe food products by prevention instead of quality inspection luning marcelis  jongen 2002  haccp is a system applied to identify known food safety hazards and is currently applied per stage in the production supply chain instead of the total production chain food safety ra comprises the scientific evaluation of known or potential adverse health effects resulting from human exposure to specific food borne hazards codex 1999  it typically uses data on the particular hazard and production chain under consideration and modelling to estimate the final likelihood of harm due to human exposure both haccp and r

In [0]:
#Reads the file.
file_content = "\n".join(df_sentences_list)
with open("input_text.txt","w") as f:
  f.write(file_content)


In [0]:
#Text embedding based on feed-forward Neural-Net Language Models with pre-built OOV. Maps from text to 128-dimensional embedding vectors
module_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
embed = hub.KerasLayer(module_url)

# Main corpus of th sentence list
corpus = df_sentences_list
corpus_embeddings = embed(corpus)

# Query sentences:

queries1 = ['Risk factors like diabetes hypertension heart disease cancer smoking status history of lung disease local climate affect?'
]
query_embeddings = embed(queries1)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences")
for query, query_embedding in zip(queries1, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("==========================Query==============================")
    print("===",query,"=====")
  

    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences


==========================Query==============================
=== diabetes hypertension heart disease cancer smoking status history of lung disease local climate =====
Score:    (Score: 0.8458) 

Paragraph:    hypertension diabetes cardiovascular disease chronic obstructive pulmonary diseases asthma and endstage renal failure 

paper_id:   5f55a576b851f033f2ea5ac413d3b3d64369ffb1 

Title:    Association Between Hypoxemia and Mortality<br>in Patients With COVID-19 

Abstract:   disease 2019 covid19 associated pneumonia
patients and methods retrospective cohort study of 140 patients with moderatetocritical covid19 associated pneumonia requiring  oxygen saturation spo 2  and other measures were tested as predictors of inhospital mortality in survival analysis
of 140 patients with covid19 associated pneumonia 514 were men with a median age of 60 years patients with spo 2 90 were older more likely to be men to have hypertension and to present with dyspnea tha